<a href="https://colab.research.google.com/github/anirbansen3027/SecureAndPrivateAI/blob/master/8_secret_sharing_and_fixed_precision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install syft

In [0]:
from torchvision import datasets, transforms
import torch as th
from torch import nn,optim
import syft as sy
import numpy as np

In [0]:
hook = sy.TorchHook(th)

W0727 19:48:46.468231 140399355094912 hook.py:98] Torch was already hooked... skipping hooking process


In [0]:
alice = sy.VirtualWorker(hook,id = "alice")
bob = sy.VirtualWorker(hook,id = "bob")
charlie = sy.VirtualWorker(hook,id = "charlie")

In [0]:
bob.add_workers([alice,charlie])
alice.add_workers([bob,charlie])
alice.add_workers([bob,alice])

W0727 19:48:46.490956 140399355094912 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0727 19:48:46.493071 140399355094912 base.py:628] Worker charlie already exists. Replacing old worker which could cause                     unexpected behavior
W0727 19:48:46.495309 140399355094912 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0727 19:48:46.496857 140399355094912 base.py:628] Worker charlie already exists. Replacing old worker which could cause                     unexpected behavior
W0727 19:48:46.499129 140399355094912 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0727 19:48:46.501047 140399355094912 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:alice #objects:13>

In [0]:
# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

In [0]:
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)
alice_data = data[2:].send(alice)
alice_target = target[2:].send(alice)

In [0]:
model = nn.Linear(2,1)

In [0]:
bobs_model = model.copy().send(bob)
alice_model = model.copy().send(alice)

In [0]:
bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
alice_opt = optim.SGD(params=alice_model.parameters(),lr=0.1)

In [0]:
for i in range(10):
  bobs_opt.zero_grad()
  bobs_pred = bobs_model(bobs_data)
  bobs_loss = ((bobs_pred-bobs_target)**2).sum()
  bobs_loss.backward()
  
  bobs_opt.step()
  
  alice_opt.zero_grad()
  alice_pred = alice_model(alice_data)
  alice_loss = ((alice_pred-alice_target)**2).sum()
  alice_loss.backward()
  
  alice_opt.step()

In [0]:
alice_grads = alice_model.get().weight.data.fix_precision().share(alice,bob,crypto_provider=charlie)
bob_grads = bobs_model.get().weight.data.fix_precision().share(alice,bob,crypto_provider=charlie)

In [0]:
z = (alice_grads + bob_grads)/2
z.get().float_precision()

tensor([[-0.1120,  0.2020]])